<a href="https://colab.research.google.com/github/rcl0511/OPENSOURCECOLABO/blob/tts/soskin_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [ ]:
!pip install flask pyngrok sentence-transformers pandas torch

In [ ]:
!ngrok config add-authtoken 2xXtc8m4a0r7QiNQBKdMplq6GHV_5RyzhHLysHjMa74EpuaHd

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
import pandas as pd
import torch
import re
from flask import Flask, request, jsonify
from pyngrok import ngrok
from sentence_transformers import SentenceTransformer, util

# 🔹 STEP 2. 모델 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('jhgan/ko-sroberta-multitask', device=device)

# 🔹 STEP 3. 데이터 불러오기
def load_answer_csv(path):
    try:
        return pd.read_csv(path, encoding='cp949')
    except:
        return pd.read_csv(path, encoding='utf-8')

df_q = pd.read_pickle('/content/drive/MyDrive/오픈소스/화상_질문_with_embedding.pkl')
df_q['embedding'] = df_q['embedding'].apply(lambda x: torch.tensor(x).to(device))

df_a = load_answer_csv('/content/drive/MyDrive/오픈소스/화상_답변.csv')

# 🔹 STEP 4. 전처리 및 키 생성
for df in [df_q, df_a]:
    df['fileName'] = df['fileName'].astype(str).str.strip().str.upper()
    df['질환_의도'] = df['disease_name'].astype(str).str.strip() + "_" + df['intention'].astype(str).str.strip()
df_a['answer'] = df_a['answer'].fillna('')

# 🔹 STEP 5. 전처리 함수
def preprocess(text):
    text = re.sub(r"[^\w\s]", "", text)
    return text.strip().lower()

# 🔹 STEP 6. 질문 유사도 기반 질환_의도 추출 (상위 3개까지)
def find_best_conditions(user_input, top_k=3):
    input_embedding = model.encode(preprocess(user_input), convert_to_tensor=True).to(device)
    similarities = [util.cos_sim(qe, input_embedding).item() for qe in df_q['embedding']]
    top_indices = torch.topk(torch.tensor(similarities), top_k).indices.tolist()
    result = []
    for idx in top_indices:
        row = df_q.iloc[idx]
        result.append({
            "question": row['question'],
            "condition_key": row['질환_의도'],
            "similarity": round(similarities[idx], 4)
        })
    return result, input_embedding

# 🔹 STEP 7. 질환_의도 기반 답변 추출
def get_best_answer(user_input):
    user_input = user_input.strip()
    if not user_input:
        return {
            "answer": "❌ 입력된 내용이 없어요. 어떤 증상이 있으신가요?",
            "matches": [],
            "top_similar_questions": []
        }

    similar_info, input_embedding = find_best_conditions(user_input)

    condition_key = similar_info[0]['condition_key']
    answer_df = df_a[df_a['질환_의도'] == condition_key]

    if answer_df.empty:
        return {
            "answer": "❌ 해당 주제에 대한 답변이 없습니다.",
            "matches": similar_info,
            "top_similar_questions": similar_info
        }

    # ✅ "응급 대처" 중심 키워드 필터
    priority_keywords = ['조치', '예방', '대응', '응급', '처치', '초기', '해결', '냉찜질', '물로 식히기', '연고', '병원']
    filtered_df = answer_df[answer_df['answer'].str.contains('|'.join(priority_keywords), na=False)]

    # ✅ 필터된 문장이 있으면 그 중에서만 선택
    if not filtered_df.empty:
        target_df = filtered_df
    else:
        # fallback - 전체 답변에서 선택
        target_df = answer_df

    answer_texts = target_df['answer'].fillna('').tolist()
    answer_embeddings = model.encode(answer_texts, convert_to_tensor=True).to(device)

    similarities = util.cos_sim(input_embedding, answer_embeddings)[0]
    best_idx = torch.argmax(similarities).item()
    best_answer = answer_texts[best_idx].strip()

    return {
        "answer": best_answer + " 더 궁금한 점이 있으신가요?",
        "matches": similar_info,
        "top_similar_questions": similar_info
    }

# 🔹 STEP 8. Flask 서버
app = Flask(__name__)
@app.route("/answer", methods=["POST"])
def answer_api():
    data = request.get_json()
    user_input = data.get("keyword", "")
    result = get_best_answer(user_input)
    return jsonify(result)  # result 안에 이미 top_similar_questions 포함됨

# 🔹 STEP 9. ngrok 연결 및 실행
public_url = ngrok.connect(5000)
print(f"✅ Flask 서버 주소: {public_url}")

app.run(port=5000)


✅ Flask 서버 주소: NgrokTunnel: "https://059a-34-16-166-76.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [08/Jun/2025 07:49:22] "POST /answer HTTP/1.1" 200 -
